In [1]:
import pandas as pd
import sqlalchemy

In [2]:
path = './data.json'

def read_dataset() -> pd.DataFrame:
    return pd.read_json(path, encoding='windows-1251')

In [3]:
df = read_dataset()

In [4]:
df.head()

,global_id,OBJ_TYPE,OnTerritoryOfMoscow,ADDRESS,SIMPLE_ADDRESS,UNOM,P0,P1,P2,P5,...,STATUS,geoData,geodata_center,L3_TYPE,L2_TYPE,P4,P6,P3,P90,P91
0,645622141,Здание,да,"Российская Федерация, город Москва, внутригоро...","Косинская улица, дом 26А",2102436,Российская Федерация,город Москва,внутригородская территория муниципальный округ...,муниципальный округ Вешняки,...,Внесён в ГКН,"{'coordinates': [[[37.8279504545784, 55.717660...","{'coordinates': [37.8281893944955, 55.71748278...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,645622142,Здание,да,"Российская Федерация, город Москва, внутригоро...","Гороховский переулок, дом 21",5783,Российская Федерация,город Москва,внутригородская территория муниципальный округ...,муниципальный округ Басманный,...,Внесён в ГКН,"{'coordinates': [[[37.6682995208392, 55.766243...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,645622174,Здание,да,"город Москва, Ленинградский проспект, дом 73А,...","Ленинградский проспект, дом 73А, строение 3",2133081,NaN,город Москва,,муниципальный округ Сокол,...,Внесён в ГКН,"{'coordinates': [[[37.5136462392231, 55.803629...",NaN,строение,NaN,NaN,NaN,NaN,NaN,NaN
3,645622205,Здание,да,"Российская Федерация, город Москва, внутригоро...","улица Вострухина, дом 5А",2128644,Российская Федерация,город Москва,внутригородская территория муниципальный округ...,муниципальный округ Рязанский,...,Внесён в ГКН,"{'coordinates': [[[37.797614164117, 55.7200933...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,645622216,Здание,да,"город Москва, улица Космонавта Волкова, дом 10...","улица Космонавта Волкова, дом 10, строение 1",3400130,NaN,город Москва,,муниципальный округ Войковский,...,Внесён в ГКН,"{'coordinates': [[[37.5139117566279, 55.815524...",NaN,строение,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
df.columns

Index(['global_id', 'OBJ_TYPE', 'OnTerritoryOfMoscow', 'ADDRESS',
       'SIMPLE_ADDRESS', 'UNOM', 'P0', 'P1', 'P2', 'P5', 'P7', 'L1_TYPE',
       'L1_VALUE', 'L2_VALUE', 'L3_VALUE', 'L4_VALUE', 'L5_VALUE', 'ADM_AREA',
       'DISTRICT', 'NREG', 'DREG', 'N_FIAS', 'D_FIAS', 'KAD_N', 'KAD_ZU',
       'KLADR', 'TDOC', 'NDOC', 'DDOC', 'ADR_TYPE', 'VID', 'SOSTAD', 'STATUS',
       'geoData', 'geodata_center', 'L3_TYPE', 'L2_TYPE', 'P4', 'P6', 'P3',
       'P90', 'P91'],
      dtype='object')

In [20]:
new_df = df[['UNOM', 'SIMPLE_ADDRESS', 'geoData', 'geodata_center']]
new_df.columns

Index(['UNOM', 'SIMPLE_ADDRESS', 'geoData', 'geodata_center'], dtype='object')

In [22]:
new_df = new_df.rename({'UNOM': 'unom', 'SIMPLE_ADDRESS': 'address', 'geoData': 'pos_geojson', 'geodata_center': 'pos_geojson_center'}, axis=1)
new_df.columns

Index(['unom', 'address', 'pos_geojson', 'pos_geojson_center'], dtype='object')

In [23]:
# new_df.loc[new_df['SIMPLE_ADDRESS'] == 'улица 800-летия Москвы, дом 5, корпус 1']

# new_df = new_df.dropna()
new_df['unom'] = new_df['unom'].astype(int)
new_df['pos_geojson'] = new_df['pos_geojson'].astype(str)
new_df['pos_geojson_center'] = new_df['pos_geojson_center'].astype(str)

In [24]:
url = 'postgresql+psycopg2://<user>:<host>@<host>:<port>/<db>' 

In [25]:
engine = sqlalchemy.create_engine(url) 

In [26]:
new_df.to_sql('addresses', engine, index=False, if_exists='replace')

/Users/varlamov/techpotion/leaders2022/infra/addresses_to_database/env/lib/python3.9/site-packages/pandas/io/sql.py:1781: SAWarning: Did not recognize type 'geometry' of column 'center_gis'
  self.meta.reflect(bind=self.connectable, only=[table_name], schema=schema)


401

In [27]:
engine.execute('''
BEGIN;

DELETE FROM addresses WHERE 
	pos_geojson = 'nan' AND pos_geojson_center = 'nan';

ALTER TABLE IF EXISTS addresses
	ADD COLUMN IF NOT EXISTS center_gis geometry;
ALTER TABLE IF EXISTS addresses
	ADD COLUMN IF NOT EXISTS center_x float;
ALTER TABLE IF EXISTS addresses
	ADD COLUMN IF NOT EXISTS center_y float;

CREATE INDEX IF NOT EXISTS addr_idx ON addresses(address);
CREATE UNIQUE INDEX IF NOT EXISTS addr_unom_idx ON addresses(unom);
CREATE INDEX IF NOT EXISTS addr_x ON addresses(center_x);
CREATE INDEX IF NOT EXISTS addr_y ON addresses(center_y);

UPDATE addresses
	SET center_gis = ST_SetSRID(ST_Centroid(ST_GeomFromGeoJSON(pos_geojson)), 4326)
	WHERE pos_geojson_center = 'nan';
	
UPDATE addresses
	SET center_gis = ST_SetSRID(ST_GeomFromGeoJSON(pos_geojson_center), 4326)
	WHERE pos_geojson_center != 'nan';

UPDATE addresses
	SET 
		center_x = ST_X(center_gis),
		center_y = ST_Y(center_gis);

COMMIT;
''')